<a href="https://colab.research.google.com/github/RachelRamirez/misclassification_matrix/blob/main/Reproducible_Misclassification_Cost_Matrix_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

After reading Keras Issues board for ways to include a cost-matrix, I found two ways provided by authors tboquet  and Isaranto.  At first I wasn't getting reproducible results, but then I realized it was because it depended on the order the neural nets were run.  I think this is largely due to the batch order re-cycling through each sequence.  I could not keep the batch order the same.  If I was able to keep it the same I believe I would get the exact same number each time.    Notice for the reproducibility example I had to seed everything, and I took out Dropout, and I set the batch size very high to minimize the variance between training.  Notice that this notebook doesn't immediately make it extremely obvious how it's reproducible.  The key is to change the order of the three functions Normal_Method, Isantro_method, and TB_Method, and you'll see that no matter what method you call first, you'll get the same numerical results each time.   Etc for 2nd and etc for 3rd. The table that follows shows the order of different functions and the first four rounded loss-values of the training data.  all methods produce the same numerical results, it just depends on the order.  

 Loss Values | is>nor>tb |	is>tb>nor |	tb>nor>is	| nor>is>tb | 
---------|-----------|-----------|-----------|-----------|
1st Method: | 	[1.472, 0.653, 0.49 , 0.344] 	| [1.472, 0.653, 0.49 , 0.344] | 	[1.472, 0.653, 0.49 , 0.344])| 	[1.472, 0.653, 0.49 , 0.344] 
2nd Method: | 	[1.53 , 0.654, 0.488, 0.389] |	[1.53 , 0.654, 0.488, 0.389] | 	[1.53 , 0.654, 0.488, 0.389] |	([1.53 , 0.654, 0.488, 0.389])
3rd Method:	| [1.519, 0.685, 0.498, 0.399] | 	[1.519, 0.685, 0.498, 0.399] | 	[1.519, 0.685, 0.498, 0.399]  | 	[1.519, 0.685, 0.498, 0.399])


### Reproducible Seeds

In [42]:
#For Reproducibility
import numpy as np
# np.random.seed(1337)  # for reproducibility

import tensorflow as tf
# tf.random.set_seed(33)

import random as python_random
# python_random.seed(4)

# https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed
tf.keras.utils.set_random_seed(342) #Possibly use next iteration if the above doesn't work


# Running more than once causes variation.  try adding this:
# Set seed value
seed_value = 56
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

print("TF version: " , tf.__version__ )
print("Keras version: " , tf.keras.__version__ )

TF version:  2.9.2
Keras version:  2.9.0


### Import rest of Library

In [43]:

# from https://github.com/keras-team/keras/issues/2115#issuecomment-204060456
# witha correction on the weighted function in the middle 

'''Train a simple deep NN on the MNIST dataset.
Get to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function  #do i still need this?
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import keras.backend as K
from itertools import product
import functools
from functools import partial

from time import sleep
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
from sklearn.metrics import confusion_matrix

## MORE REPEATABILITY STUFF NEEDED - If theres a way to update this to V2 of Tensorflow great, otherwise I had to use TF 1.0 code
# 5. Configure a new global `tensorflow` session (https://stackoverflow.com/questions/50659482/why-cant-i-get-reproducible-results-in-keras-even-though-i-set-the-random-seeds)
# from keras import backend as K


#I believe thecode below is to help things be repeatable each time different sections in my google colab notebook execute
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

### Define batch, epochs, and format data

In [44]:
batch_size = 5000 # I originally had it very  high batch size to reduce the variation in the data each batch and hope it makes the model training more nearly identical which it did, then i bring it back down to something reasonable to get better results training the NN
nb_classes = 10
nb_epoch = 4

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()


X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


60000 train samples
10000 test samples


### Define weighted_categorical_crossentropy()

In [45]:

# https://github.com/keras-team/keras/issues/2115#issuecomment-207765342

def w_categorical_crossentropy(y_true, y_pred, weights):
    nb_cl = len(weights)
    final_mask = K.zeros_like(y_pred[:, 0])
    y_pred_max = K.max(y_pred, axis=1)
    y_pred_max = K.expand_dims(y_pred_max, 1)
    y_pred_max_mat = K.equal(y_pred, y_pred_max)
 
    for c_t, c_p in product(range(nb_cl), range(nb_cl)):
        final_mask += (K.cast(weights[c_t, c_p],K.floatx()) * K.cast(y_pred_max_mat[:, c_p] ,K.floatx())* K.cast(y_true[:, c_t],K.floatx()))
    
    # result = K.categorical_crossentropy(y_true, y_pred)*final_mask
    # tf.print(result, "Show Result of CE * Final_Mask")  #this was basically useless to display, and it showed like, 500 lines of print statements each epoch

    return K.categorical_crossentropy(y_true, y_pred)*final_mask   #I changed the order of y_true and y_pred




### Define Same Model but use normal Categorical CrossEntropy with no extra cost-matrix of Weights

In [46]:
def normal_method():

  model = Sequential()
  model.add(Dense(512, input_shape=(784,) ,kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model.add(Activation('relu'))
  # model.add(Dropout(0.2))
  model.add(Dense(512, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model.add(Activation('relu'))
  # model.add(Dropout(0.2))
  model.add(Dense(10, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model.add(Activation('softmax'))

  rms = RMSprop()
  # model.compile(loss=ncce, optimizer=rms)
  model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=rms, metrics='categorical_accuracy', )

  #add early_stop to prevent overfittings
  # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  model_history = model.fit(X_train, Y_train,
            batch_size=batch_size, epochs=nb_epoch, verbose=0,
            validation_data=(X_test, Y_test),shuffle=False, use_multiprocessing=False
            # callbacks = [callback])
            )

  
  # model.evaluate(X_test, Y_test, verbose=1)  # I know this isn't the typical use of train/val/test sets, please dont' comment on that
  
  #Predict
  y_prediction = model.predict(X_test)
  y_prediction  = np.argmax(y_prediction, axis=1)
  # Y_prediction = np_utils.to_categorical(y_prediction, nb_classes) #If I want to do SparseCategoricalAccuracy

  #Create confusion matrix and normalizes it over predicted (columns)
  # result = confusion_matrix(y_test, y_prediction , normalize='pred')  #if I want percentages instead of raw counts

  
  cm = confusion_matrix(y_test, y_prediction)
  cm = pd.DataFrame(cm, range(10),range(10))

  #This shows a pretty confusion matrix which I don't neeed to show right now
  # plt.figure(figsize = (10,10))
  # sns.heatmap(cm, annot=True, annot_kws={"size": 12}) # font size
  # plt.show()
  cm_normal = cm

  return model_history


Repeatable!

Epoch 1/4
6/6 [==============================] - 3s 491ms/step - **loss: 1.5304** - categorical_accuracy: 0.5081 - val_loss: 0.7234 - val_categorical_accuracy: 0.8170
Epoch 2/4
6/6 [==============================] - 3s 467ms/step - loss: **0.6541** - categorical_accuracy: 0.8048 - val_loss: 0.5255 - val_categorical_accuracy: 0.8161
Epoch 3/4
6/6 [==============================] - 3s 467ms/step - loss: 0.4883 - categorical_accuracy: 0.8496 - val_loss: 0.4127 - val_categorical_accuracy: 0.8734
Epoch 4/4
6/6 [==============================] - 3s 475ms/step - loss: 0.3893 - categorical_accuracy: 0.8840 - val_loss: 0.3287 - val_categorical_accuracy: 0.9073
313/313 [==============================] - 1s 3ms/step - loss: 0.3287 - categorical_accuracy: 0.9073
313/313 [==============================] - 1s 2ms/step

### Define 2nd Model using loss= Weighted_Cross_Entropy and "normal" cost matrix

In [47]:
 #what does ncce stand for? non-uniform cost cross entropy? tboquet on Keras Issues Team named it this way March 31 2016

def tb_method():
  w_array = np.ones((10,10))
  # w_array[9, 7] = 1.5
  # w_array = w_array - np.eye(10)
  # print("W_array:  ", w_array)

  # ncce = partial(w_categorical_crossentropy, weights=np.ones((10,10)))


  ncce = partial(w_categorical_crossentropy, weights=w_array)
  ncce.__name__ ='w_categorical_crossentropy'

  model2 = Sequential()
  model2.add(Dense(512, input_shape=(784,) ,kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model2.add(Activation('relu'))
  # model2.add(Dropout(0.2))
  model2.add(Dense(512, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model2.add(Activation('relu'))
  # model2.add(Dropout(0.2))
  model2.add(Dense(10, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model2.add(Activation('softmax'))


  rms = RMSprop()

  model2.compile(loss=ncce, optimizer=rms,  metrics='categorical_accuracy',)
  # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  model2_history = model2.fit(X_train, Y_train,
            batch_size=batch_size, epochs=nb_epoch, verbose=0,
            validation_data=(X_test, Y_test), shuffle=False, use_multiprocessing=False
            #callbacks = [callback])
  )

  # len(history.history['loss'])

  # print('Test score:', score[0])
  # print('Test accuracy:', score[1])

  from sklearn.metrics import confusion_matrix

  #Predict
  y_prediction = model2.predict(X_test)
  y_prediction  = np.argmax(y_prediction, axis=1)
  # Y_prediction = np_utils.to_categorical(y_prediction, nb_classes)

  #Create confusion matrix and normalizes it over predicted (columns)
  # result = confusion_matrix(y_test, y_prediction , normalize='pred')

  cm2 = confusion_matrix(y_test, y_prediction)
  cm2 = pd.DataFrame(cm2, range(10),range(10))
  # plt.figure(figsize = (10,10))
  # sns.heatmap(cm2, annot=True, annot_kws={"size": 12}) # font size
  # plt.show()
  
  cm_using_weighted = cm2

  return model2_history

REPEATABLE - Epoch 1/4
6/6 [==============================] - 11s 1s/step - loss: **1.4722** - categorical_accuracy: 0.5274 - val_loss: 0.7470 - val_categorical_accuracy: 0.7868
Epoch 2/4
6/6 [==============================] - 5s 773ms/step - loss: **0.6526** - categorical_accuracy: 0.7996 - val_loss: 0.5025 - val_categorical_accuracy: 0.8361
Epoch 3/4
6/6 [==============================] - 4s 627ms/step - loss: 0.4899 - categorical_accuracy: 0.8417 - val_loss: 0.4000 - val_categorical_accuracy: 0.8866
Epoch 4/4
6/6 [==============================] - 3s 478ms/step - loss: 0.3440 - categorical_accuracy: 0.9026 - val_loss: 0.3035 - val_categorical_accuracy: 0.9118
313/313 [==============================] - 1s 3ms/step

Not Repeatable upon Re-Running (not Restart+Rerun) 

Epoch 1/4
6/6 [==============================] - 6s 662ms/step - loss: 1.5140 - categorical_accuracy: 0.5150 - val_loss: 0.8487 - val_categorical_accuracy: 0.7004
Epoch 2/4
6/6 [==============================] - 3s 473ms/step - loss: 0.6816 - categorical_accuracy: 0.7931 - val_loss: 0.4447 - val_categorical_accuracy: 0.8790
Epoch 3/4
6/6 [==============================] - 3s 475ms/step - loss: 0.4923 - categorical_accuracy: 0.8434 - val_loss: 0.3447 - val_categorical_accuracy: 0.9046
Epoch 4/4
6/6 [==============================] - 3s 475ms/step - loss: 0.3922 - categorical_accuracy: 0.8845 - val_loss: 0.3259 - val_categorical_accuracy: 0.9045
313/313 [==============================] - 1s 3ms/step

Not Repeatable upon Re-Running:

> Epoch 1/4
6/6 [==============================] - 6s 664ms/step - loss: 1.4762 - categorical_accuracy: 0.5231 - val_loss: 0.7467 - val_categorical_accuracy: 0.7548
Epoch 2/4
6/6 [==============================] - 3s 480ms/step - loss: 0.6470 - categorical_accuracy: 0.7904 - val_loss: 0.4843 - val_categorical_accuracy: 0.8507
Epoch 3/4
6/6 [==============================] - 3s 483ms/step - loss: 0.4990 - categorical_accuracy: 0.8395 - val_loss: 0.4301 - val_categorical_accuracy: 0.8638
Epoch 4/4
6/6 [==============================] - 3s 552ms/step - loss: 0.3866 - categorical_accuracy: 0.8798 - val_loss: 0.4996 - val_categorical_accuracy: 0.8318
313/313 [==============================] - 1s 4ms/step


In [48]:
# cm_using_weighted

### Trying a user's suggestion for a more vectorized appraoch Model 3

In [49]:
# from https://github.com/keras-team/keras/issues/2115#issuecomment-815825633 from Isaranto

def weighted_categorical_crossentropy_new(y_true, y_pred, weights):
          idx1 = K.argmax(y_pred, axis=1)
          idx2 = K.argmax(y_true, axis=1)
          mask = tf.gather_nd(weights, tf.stack((idx1, idx2), -1))
          return K.categorical_crossentropy(y_true, y_pred) * mask

In [50]:
 #what does ncce stand for?

 
def isaranto_method():
  w_array = np.ones((10,10))
  # w_array[9, 7] = 1.5
  # w_array = w_array - np.eye(10)
  # print("W_array:  ", w_array)

  weighted_list = w_array.tolist()

  wcce = partial(weighted_categorical_crossentropy_new, weights=weighted_list)
  wcce.__name__ ='w_categorical_crossentropy'

  model3 = Sequential()
  model3.add(Dense(512, input_shape=(784,), kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('relu'))
  # model3.add(Dropout(0.2))
  model3.add(Dense(512, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('relu'))
  # model3.add(Dropout(0.2))
  model3.add(Dense(10,kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('softmax'))

  rms = RMSprop()

  model3.compile(loss=wcce, optimizer=rms,  metrics='categorical_accuracy',)
  # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  model3_history = model3.fit(X_train, Y_train,
            batch_size=batch_size, epochs=nb_epoch, verbose=0,
            validation_data=(X_test, Y_test), shuffle=False, use_multiprocessing=False
            # callbacks = [callback]
            )

  
  # print('Test score:', score[0])
  # print('Test accuracy:', score[1])


  #Predict
  y_prediction = model3.predict(X_test)
  y_prediction  = np.argmax(y_prediction, axis=1)
  # Y_prediction = np_utils.to_categorical(y_prediction, nb_classes)

  #Create confusion matrix and normalizes it over predicted (columns)
  # result = confusion_matrix(y_test, y_prediction , normalize='pred')

  

  cm3 = confusion_matrix(y_test, y_prediction)
  cm3 = pd.DataFrame(cm3, range(10),range(10))
  # plt.figure(figsize = (10,10))
  # cm3
  # sns.heatmap(cm2, annot=True, annot_kws={"size": 12}) # font size
  # plt.show()

  # cm_using_weighted_new = cm3

  return model3_history

Repeatable, AND This loss function looks exactly like the Normal one without specified costs!
Epoch 1/4
6/6 [==============================] - 3s 498ms/step - loss: **1.5185** - categorical_accuracy: 0.5047 - val_loss: 0.8206 - val_categorical_accuracy: 0.7422
Epoch 2/4
6/6 [==============================] - 3s 469ms/step - loss:**0.6851** categorical_accuracy: 0.7891 - val_loss: 0.5890 - val_categorical_accuracy: 0.7968
Epoch 3/4
6/6 [==============================] - 3s 469ms/step - loss: 0.4974 - categorical_accuracy: 0.8435 - val_loss: 0.3986 - val_categorical_accuracy: 0.8833
Epoch 4/4
6/6 [==============================] - 3s 461ms/step - loss: 0.3994 - categorical_accuracy: 0.8785 - val_loss: 0.3041 - val_categorical_accuracy: 0.9101
313/313 [==============================] - 1s 2ms/step

In [51]:
# cm_using_weighted_new 

In [52]:
# cm_normal

### Comparing the loss history across the three models

In [53]:
normal_model_history  = normal_method()
np.round(normal_model_history.history['loss'],3)  

313/313 [==============================] - 1s 4ms/step


array([1.073, 0.452, 0.305, 0.311])

In [54]:
is_model_history = isaranto_method()
np.round(is_model_history.history['loss'],3)

313/313 [==============================] - 1s 2ms/step


array([1.094, 0.435, 0.339, 0.257])

In [55]:
tb_model_history = tb_method()
np.round(tb_model_history.history['loss'],3)  

313/313 [==============================] - 1s 3ms/step


array([1.105, 0.448, 0.346, 0.267])

#### Return the loss history of Normal, TB, Isantro (in that order for putting ni a table to see differences, but the order ran is as given above)

### Re-Run First Model with no Weights to see if its still the same

In [56]:
# # # The results of running the model again as model1 were not identical to model earlier.  So I am trying to put the re-initialized seeds here to see if that makes a difference
 


# model1 = Sequential()
# model1.add(Dense(512, input_shape=(784,) ,kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
# model1.add(Activation('relu'))
# # model1.add(Dropout(0.2))
# model1.add(Dense(512, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
# model1.add(Activation('relu'))
# # model1.add(Dropout(0.2))
# model1.add(Dense(10, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
# model1.add(Activation('softmax'))

# rms = RMSprop()
# # model1.compile(loss=ncce, optimizer=rms)
# model1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=rms, metrics='categorical_accuracy', )

# #add early_stop to prevent overfittings
# # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# model1_history = model1.fit(X_train, Y_train,
#           batch_size=batch_size, epochs=nb_epoch, verbose=0,
#           validation_data=(X_test, Y_test),shuffle=False, use_multiprocessing=False
#           # callbacks = [callback])
#           )

 
# model1.evaluate(X_test, Y_test, verbose=1)
 


# # Predict
# y_prediction = model1.predict(X_test)
# y_prediction  = np.argmax(y_prediction, axis=1)
# # Y_prediction = np_utils.to_categorical(y_prediction, nb_classes)

# #Create confusion matrix and normalizes it over predicted (columns)
# # result = confusion_matrix(y_test, y_prediction , normalize='pred')

 
# cm1 = confusion_matrix(y_test, y_prediction)
# cm1 = pd.DataFrame(cm1, range(10),range(10))
# # plt.figure(figsize = (10,10))

# # sns.heatmap(cm1, annot=True, annot_kws={"size": 12}) # font size
# # plt.show()

# cm_normal_replicate = cm1




Repeatable - Epoch 1/4
6/6 [==============================] - 5s 620ms/step - loss: 1.4564 - categorical_accuracy: 0.5507 - val_loss: 0.8112 - val_categorical_accuracy: 0.7332
Epoch 2/4
6/6 [==============================] - 3s 588ms/step - loss: 0.7296 - categorical_accuracy: 0.7690 - val_loss: 0.4210 - val_categorical_accuracy: 0.8938
Epoch 3/4
6/6 [==============================] - 4s 777ms/step - loss: 0.4762 - categorical_accuracy: 0.8580 - val_loss: 0.3746 - val_categorical_accuracy: 0.8962
Epoch 4/4
6/6 [==============================] - 3s 584ms/step - loss: 0.3397 - categorical_accuracy: 0.9036 - val_loss: 0.4469 - val_categorical_accuracy: 0.8487
313/313 [==============================] - 1s 3ms/step - loss: 0.4469 - categorical_accuracy: 0.8487
313/313 [==============================] - 1s 3ms/step

Epoch 1/4
6/6 [==============================] - 3s 501ms/step - loss: **1.5185** - categorical_accuracy: 0.5047 - val_loss: 0.8206 - val_categorical_accuracy: 0.7422
Epoch 2/4
6/6 [==============================] - 3s 472ms/step - loss: **0.6851** - categorical_accuracy: 0.7891 - val_loss: 0.5890 - val_categorical_accuracy: 0.7968
Epoch 3/4
6/6 [==============================] - 3s 474ms/step - loss: 0.4974 - categorical_accuracy: 0.8435 - val_loss: 0.3986 - val_categorical_accuracy: 0.8833
Epoch 4/4
6/6 [==============================] - 3s 477ms/step - loss: 0.3994 - categorical_accuracy: 0.8785 - val_loss: 0.3041 - val_categorical_accuracy: 0.9101
313/313 [==============================] - 1s 3ms/step - loss: 0.3041 - categorical_accuracy: 0.9101
313/313 [==============================] - 1s 3ms/step

Not Repeatable - 
Epoch 1/4
6/6 [==============================] - 3s 496ms/step - loss: **1.4722**- categorical_accuracy: 0.5274 - val_loss: 0.7470 - val_categorical_accuracy: 0.7868
Epoch 2/4
6/6 [==============================] - 3s 464ms/step - loss: **0.6526** categorical_accuracy: 0.7996 - val_loss: 0.5025 - val_categorical_accuracy: 0.8361
Epoch 3/4
6/6 [==============================] - 4s 623ms/step - loss: 0.4899 - categorical_accuracy: 0.8417 - val_loss: 0.4000 - val_categorical_accuracy: 0.8866
Epoch 4/4
6/6 [==============================] - 3s 464ms/step - loss: 0.3440 - categorical_accuracy: 0.9026 - val_loss: 0.3035 - val_categorical_accuracy: 0.9118
313/313 [==============================] - 1s 3ms/step - loss: 0.3035 - categorical_accuracy: 0.9118
313/313 [==============================] - 1s 3ms/step

Not Repeatable - Epoch 1/4
6/6 [==============================] - 4s 497ms/step - **loss: 1.4564**- categorical_accuracy: 0.5507 - val_loss: 0.8112 - val_categorical_accuracy: 0.7332
Epoch 2/4
6/6 [==============================] - 3s 482ms/step - loss: **0.7296** - categorical_accuracy: 0.7690 - val_loss: 0.4210 - val_categorical_accuracy: 0.8939
Epoch 3/4
6/6 [==============================] - 3s 487ms/step - loss: 0.4762 - categorical_accuracy: 0.8579 - val_loss: 0.3743 - val_categorical_accuracy: 0.8962
Epoch 4/4
6/6 [==============================] - 3s 473ms/step - loss: 0.3394 - categorical_accuracy: 0.9038 - val_loss: 0.4460 - val_categorical_accuracy: 0.8490
313/313 [==============================] - 1s 3ms/step - loss: 0.4460 - categorical_accuracy: 0.8490
313/313 [==============================] - 1s 3ms/step

In [57]:
# model1_history.history['loss']

In [58]:
# for i in range(0,len(model1_history.history['loss'])):
#   print(model1_history.history['loss'][i] - model_history.history['loss'][i])

In [59]:
# cm1 - cm

### Other users scratch work that I haven't incorporated or may not be useful

In [60]:
# import numpy as np
# from keras.api._v2 import keras as tk
# import tensorflow as tf
# from keras.utils import losses_utils
# import typing as t


# class CostSensitiveLoss(tk.losses.Loss):

#     def __init__(
#         self,
#         cost_matrix: t.List, loss: tk.losses.Loss,
#     ):
#         super().__init__(reduction=loss.reduction, name=loss.name)
#         self.loss = loss
#         self.cost_matrix = cost_matrix
#         self._cost_matrix = tf.constant(cost_matrix, dtype=tf.float32)

#     @classmethod
#     def from_config(cls, config):
#         config['loss'] = tk.losses.deserialize(config['loss'])
#         return cls(**config)

#     def get_config(self):
#         return {
#             'cost_matrix': self.cost_matrix,
#             'loss': tk.losses.serialize(self.loss),
#             'reduction': self.reduction, 'name': self.name
#         }
    
#     def call(self, y_true, y_pred):
#         # if y_true is one hot encoded then get integer indices
#         if y_true.ndim == 1:
#             y_true_index = y_true
#         elif y_true.ndim == 2:
#             y_true_index = tf.argmax(y_true, axis=1)
#         else:
#             raise Exception(f"`y_true.ndim` {y_true.ndim} not supported")
        
#         # get cost for batch
#         cost_for_batch = tf.nn.embedding_lookup(self._cost_matrix, y_true_index)
#         cost_for_batch *= y_pred
#         cost_for_batch = tf.reduce_sum(cost_for_batch, axis=1)
        
#         # get loss
#         return self.loss(y_true, y_pred, cost_for_batch)


# if __name__ == '__main__':
#     # for debug purpose I have kept 'none' you can
#     # safely use other options like 'sum', 'auto'
#     _loss = tk.losses.MeanAbsoluteError(reduction='none')
    
#     # some cost matrices the first cost matrix is the case when you are
#     # not using cost sensitive weights
#     _cs_loss_1 = CostSensitiveLoss(
#         cost_matrix=[[1, 1, 1], [1, 1, 1], [1, 1, 1], ],
#         loss=_loss
#     )
#     _cs_loss_2 = CostSensitiveLoss(
#         cost_matrix=[[1, 2, 2], [4, 1, 4], [8, 8, 1], ],
#         loss=_loss
#     )
#     _cs_loss_3 = CostSensitiveLoss(
#         cost_matrix=[[1, 4, 8], [2, 1, 8], [2, 4, 1], ],
#         loss=_loss
#     )
#     _y_true = np.asarray(
#         [
#             [1, 0, 0],
#             [0, 1, 0],
#             [0, 0, 1],
#             [1, 0, 0],
#             [0, 1, 0],
#             [0, 0, 1],
#             [1, 0, 0],
#             [0, 1, 0],
#             [0, 0, 1],
#         ]
#     )
#     _y_pred = np.asarray(
#         [
#             [0.8, 0.1, 0.1],
#             [0.1, 0.8, 0.1],
#             [0.1, 0.1, 0.8],
            
#             [0.1, 0.8, 0.1],
#             [0.1, 0.1, 0.8],
#             [0.8, 0.1, 0.1],
            
#             [0.1, 0.1, 0.8],
#             [0.8, 0.1, 0.1],
#             [0.1, 0.8, 0.1],
#         ]
#     )
#     print("loss ........................")
#     print(_loss(_y_true, _y_pred).numpy())
    
#     print("cs_loss_1 ...................")
#     print(_cs_loss_1(_y_true, _y_pred).numpy())
    
#     print("cs_loss_2 ...................")
#     print(_cs_loss_2(_y_true, _y_pred).numpy())
    
#     print("cs_loss_3 ...................")
#     print(_cs_loss_3(_y_true, _y_pred).numpy())

In [61]:
# import tensorflow.keras.backend as K
# from tensorflow.keras.losses import CategoricalCrossentropy


# class WeightedCategoricalCrossentropy(CategoricalCrossentropy):
    
#     def __init__(self, cost_mat, name='weighted_categorical_crossentropy', **kwargs):
#         assert cost_mat.ndim == 2
#         assert cost_mat.shape[0] == cost_mat.shape[1]
        
#         super().__init__(name=name, **kwargs)
#         self.cost_mat = K.cast_to_floatx(cost_mat)
    
#     def __call__(self, y_true, y_pred, sample_weight=None):
#         assert sample_weight is None, "should only be derived from the cost matrix"
      
#         return super().__call__(
#             y_true=y_true,
#             y_pred=y_pred,
#             sample_weight=get_sample_weights(y_true, y_pred, self.cost_mat),
#         )


# def get_sample_weights(y_true, y_pred, cost_m):
#     num_classes = len(cost_m)

#     y_pred.shape.assert_has_rank(2)
#     y_pred.shape[1:].assert_is_compatible_with(num_classes)
#     y_pred.shape.assert_is_compatible_with(y_true.shape)

#     y_pred = K.one_hot(K.argmax(y_pred), num_classes)

#     y_true_nk1 = K.expand_dims(y_true, 2)
#     y_pred_n1k = K.expand_dims(y_pred, 1)
#     cost_m_1kk = K.expand_dims(cost_m, 0)

#     sample_weights_nkk = cost_m_1kk * y_true_nk1 * y_pred_n1k
#     sample_weights_n = K.sum(sample_weights_nkk, axis=[1, 2])

#     return sample_weights_n
# # Usage:

# model.compile(loss=WeightedCategoricalCrossentropy(cost_matrix), ...)